In [ ]:
from model_gens.gru_model_gen import GRUModel
from model_gens.lstm_model_gen import LSTMModel
from model_gens.meta_model_gen import MetaModel
import tensorflow as tf # type: ignore

# # deanreadhead99@gmail.com
# # +1-780-222-1086
# #clack

# Enable memory growth for GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
market_predictor_lstm = LSTMModel()
market_predictor_gru = GRUModel()
market_predictor_meta = MetaModel()

In [ ]:
data_path = '/home/amar/Documents/Projects/MarketPredictor/Datas/NEW_XAUUSD/new/^XAUUSD_combined.csv'

In [ ]:
# Use LSTM or GRU model to process data
data = market_predictor_lstm.load_data(data_path)

In [5]:
preprocessed_data = market_predictor_lstm.preprocess_data(data)

In [6]:
X, y = market_predictor_lstm.prepare_dataset(preprocessed_data)

In [7]:
X_train, X_test, y_train, y_test = market_predictor_lstm.split_data(X, y)

In [8]:
market_predictor_lstm.load_model(model_name='models/lstm/XAUUSD_combined_lstm')
market_predictor_gru.load_model(model_name='models/gru/XAUUSD_combined_gru')

2024-07-03 16:52:33.403816: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-03 16:52:33.404124: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-03 16:52:33.404275: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

<Sequential name=sequential, built=True>

In [9]:
market_predictor_lstm.load_scaler(
    scaler_filename='/home/amar/Documents/Projects/MarketPredictor/models/scalers/lstm/XAUUSD_lstm_scaler.pkl'
)
market_predictor_gru.load_scaler(
    scaler_filename='/home/amar/Documents/Projects/MarketPredictor/models/scalers/gru/XAUUSD_gru_scaler.pkl'
)

In [10]:
X_train_meta = market_predictor_meta.generate_meta_features(
    lstm_model=market_predictor_lstm.model, gru_model=market_predictor_gru.model, X=X_train, gru_scaler=market_predictor_gru.scaler, lstm_scaler=market_predictor_lstm.scaler)
X_test_meta = market_predictor_meta.generate_meta_features(
    market_predictor_lstm.model, market_predictor_gru.model, X_test, market_predictor_gru.scaler, market_predictor_lstm.scaler)

2024-07-03 16:52:34.294432: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906


618/618 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
618/618 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [11]:
market_predictor_meta.train_model(X_train_meta, y_train)

GradientBoostingRegressor(learning_rate=0.01, n_estimators=1000)

In [12]:
market_predictor_meta.save_meta_model('models/meta/XAUUSD_combined_meta.pkl')

In [13]:
market_predictor_meta.save_scaler_and_shape(
    '/home/amar/Documents/Projects/MarketPredictor/models/scalers/meta/XAUUSD_combined_meta_scaler.pkl',
    '/home/amar/Documents/Projects/MarketPredictor/models/shapes/meta/XAUUSD_combined_meta_shape.pkl')